In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/Shareddrives/My\ paper

/content/drive/Shareddrives/My paper


In [ ]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from collections import Counter

In [ ]:
member = pd.read_csv('dataset/91APP_MemberData.csv')

In [ ]:
order = pd.read_csv('dataset/91APP_OrderData.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
order['TradesDateTime'] = pd.to_datetime(order['TradesDateTime'])
order = order.sort_values(by='TradesDateTime')
order = order[(order['Status']=='Finish')|(order['Status']=='Return')]

In [ ]:
member['RegisterDateTime'] = pd.to_datetime(member['RegisterDateTime'])
member = member.sort_values(by='RegisterDateTime')
member = member.drop_duplicates(subset=['MemberID'], keep=False)

In [ ]:
print(len(member['MemberID'].unique()), len(order['MemberID'].unique()))

377132 225409


## EDA: 2020.05~2021.04 營收狀況

In [ ]:
memberIDs = list(member[member['RegisterDateTime']<'2020-05-01']['MemberID'])

In [ ]:
order2020 = order[(order['TradesDateTime']<'2021-05-01')&(order['TradesDateTime']>='2020-05-01')&(order['MemberID'].isin(memberIDs))]

In [ ]:
order2020['TotalSalesAmount'].sum()

92097987.2

In [ ]:
pd.DataFrame(order2020.groupby(by=['MemberID']).sum()['TotalSalesAmount']).to_csv('Net Spending of active members after 2020.05.csv')

## 轉換 UID 欄位

In [ ]:
def add_uid_col(data):
  df = data.copy()
  df['uid'] = df['MemberID'].str.replace('+', '%2B')
  df['uid'] = df['uid'].str.replace('/', '%2F')
  df['uid'] = df['uid'].str.replace('=', '%3D')

  return df['uid']

In [ ]:
member['uid'] = add_uid_col(member)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


## 讀取整理好的行為資料以取得 MemberID

In [ ]:
behaviors = pd.read_csv('./Customer Value Prediction/TABLES/all behaviors data.csv')

In [ ]:
len(behaviors['uid_filled'].unique())

119469

In [ ]:
order2019 = order[order['TradesDateTime']<'2020-05-01']

In [ ]:
member_data = member[member['uid'].isin(behaviors['uid_filled'])]

In [ ]:
member_data = member_data[member_data['MemberID'].isin(order2019['MemberID'])]

In [ ]:
uids = list(member_data['uid'].unique())
memberIDs = list(member_data['MemberID'].unique())

In [ ]:
print(len(memberIDs), len(uids))

83931 83931


## 開始產生 label data

In [ ]:
label_data = order2020[(order2020['MemberID'].isin(memberIDs))].sort_values(by='TradesDateTime')

In [ ]:
label_data = pd.DataFrame(label_data.groupby(by=['MemberID']).sum()['TotalSalesAmount'])

In [ ]:
label_data['Active'] = [1] * len(label_data)
label_data['VIP'] = [0]*len(label_data)
label_data.loc[label_data['TotalSalesAmount']>=2480, 'VIP'] = 1

In [ ]:
label_data.describe()

,TotalSalesAmount,Active,VIP
count,21102.000000,21102.0,21102.000000
mean,4364.419828,1.0,0.530661
std,6212.324696,0.0,0.499071
min,-1680.000000,1.0,0.000000
25%,1580.000000,1.0,0.000000
50%,2710.000000,1.0,1.000000
75%,4787.000000,1.0,1.000000
max,181428.000000,1.0,1.000000


In [ ]:
Counter(label_data['VIP'])

Counter({0: 9904, 1: 11198})

In [ ]:
label_data = pd.merge(member_data[['MemberID']], label_data, on='MemberID', how='left').fillna(0)

In [ ]:
label_data.to_csv('Label Data.csv', index=False)

In [ ]:
label_data.describe()

,TotalSalesAmount,Active,VIP
count,83931.000000,83931.000000,83931.000000
mean,1097.305968,0.251421,0.133419
std,3645.243457,0.433832,0.340029
min,-1680.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,1.000000,0.000000
max,181428.000000,1.000000,1.000000


In [ ]:
order_count = pd.DataFrame(order2020[(order2020['MemberID'].isin(memberIDs))].groupby(by='MemberID').size(), columns=['Count'])

In [ ]:
order_count[order_count['Count']==2]

,Count
MemberID,
+/Vntu+b6rUT+HmguVK+Ge6F6C31WtXF42aDdFKXwFU=,2
+1pQmxnIoN77XQtll5laD8VW7oHUSzhV2v2WJLC/bvk=,2
+2L3qeMLfbpnoT09K00DdiGfrvTEiK6grWnkvFlAiw0=,2
+455at2Kp+knZz5Y+vzquyrgdeS4c1IOJWz6QiL+51Y=,2
+4619tMIZpfXFAz8Tb7OivwAfdOBxIlnXZXEsvNxBww=,2
...,...
zxHYgpe57mJGir8/Q6Vu4+CBVxG1U1ytIalVpeZGiaQ=,2
zxfBGHdTzSmnQRBBn2EWtin5e9YgHCCNmMsTCCX7Lbs=,2
zxjVQqQsHkkJevOxObiJ3NKxSKtTPnuWyUA5NyHqH38=,2
